In [1]:
# recall df
%store -r w_df
%store -r w_test
%store -r w_train

# train and test that consider only variable temperature 
%store -r train_t
%store -r test_t

no stored variable or alias w_df
no stored variable or alias w_test
no stored variable or alias w_train
no stored variable or alias train_t
no stored variable or alias test_t


## PACAKES

In [2]:
import numpy as np
import pandas as pd
import statsmodels.datasets as smd
import sklearn
import scipy.stats as stats
from scipy import signal
from datetime import datetime
from matplotlib.dates import DateFormatter
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn-v0_8')


# Specific importing
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import STL
from sklearn.linear_model import LinearRegression
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, root_mean_squared_error
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.api import VAR
from statsmodels.tsa.statespace.varmax import VARMAX

#! pip install pmdarima
from pmdarima.arima import auto_arima

import warnings
warnings.filterwarnings("ignore")

## 1) Seasonal ARIMA

A seasonal ARIMA model (SARIMA) is made by including additional seasonal terms in the models we have seen so far.

These are generally written as ARIMA(p,d,q)(P,D,Q)m where the first part (p,d,q) refers to the non-seasonal component, while the second one (P,D,Q)m to the seasonal component. The term m refers to the seasonality (seasonal period, e.g., for annual data m = 12, quarterly data m = 4). This is also indicated as S (or s).

CF and PACF plots can help in identifying not only the non-seasonal component but also the seasonal part of an AR or MA model by looking at the seasonal lags of the PACF and ACF. For example:

* ARIMA(0,0,0)(0,0,1)12: 1) a significant spike at lag 12 in the ACF but no other significant spikes; 2) exponential decay in the seasonal lags of the PACF (i.e., at lags 12, 24, 36);
* ARIMA(0,0,0)(1,0,0)12: 1) exponential decay in the seasonal lags of the ACF; 2) a single significant spike at lag 12 in the PACF.